In [1]:
import cv2
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader

## AP


In [3]:
cd E:\MAJOR_project_2\Dataset\BUU_LSPINE_400\BUU_LSPINE_400\YOLO

E:\MAJOR_project_2\Dataset\BUU_LSPINE_400\BUU_LSPINE_400\YOLO


In [4]:
#load YAML
with open('yolo_yaml.yaml', mode = 'r') as f:
    ap_data_yaml = yaml.load(f, Loader = SafeLoader)
    
labels = ap_data_yaml['names']
print(labels)

['APL1', 'APL2', 'APL3', 'APL4', 'APL5']


In [5]:
cd E:\MAJOR_project_2\Dataset\BUU_LSPINE_400\BUU_LSPINE_400\YOLO\total_model

E:\MAJOR_project_2\Dataset\BUU_LSPINE_400\BUU_LSPINE_400\YOLO\total_model


In [6]:
#load YOLO model
yolo = cv2.dnn.readNetFromONNX("best.onnx")
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [7]:
#load image
img = cv2.imread(r"E:\MAJOR_project_2\Dataset\BUU_LSPINE_400\BUU_LSPINE_400\AP\0001-F-037Y0.jpg")
image = img.copy()
row, col, d = image.shape

#get the yolo prediction from the image
#step-1: convert image into square image(array)
max_rc = max(row, col)
input_image = np.zeros((max_rc, max_rc, 3), dtype = np.uint8)
input_image[0:row, 0:col] = image

#step-2: get prediction from the square array
INPUT_WH_YOLO = 640
blob = cv2.dnn.blobFromImage(input_image, 1/255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB = True, crop = False)
yolo.setInput(blob)
preds = yolo.forward() # detection or prediction from yolo



# cv2.imshow('image', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [8]:
print(preds.shape)

(1, 25200, 10)


In [9]:
#non maximum supression
#step1: filter detection based on confidence(0.3) and probability score (0.2)
detections = preds[0]
boxes = []
confidences = []
classes = []

#width and height of image(input_image)
image_w, image_h = input_image.shape[0:2]
x_factor = image_w/INPUT_WH_YOLO
y_factor = image_h/INPUT_WH_YOLO

for i in range(len(detections)):
    row = detections[i]
    confidence = row[4]
    
    if confidence > 0.4:
        class_score = row[5:].max() #max probability from 5 objects
        class_id = row[5:].argmax()
        
        if class_score > 0.25:
            cx, cy, w, h = row[0:4]
            #construct bounding box from four values
            #left, top, width and height
            left = int((cx- 0.5*w)*x_factor)
            top = int((cy - 0.5*h)*y_factor)
            width = int(w*x_factor)
            height = int(h*y_factor)
            
            box = np.array([left, top, width, height])
            
            #append values into list
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)

#clean
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

#NMS
index = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.25, 0.4).flatten()


In [10]:
index

array([65, 22, 10, 13, 62])

In [11]:
#draw bounding box
for ind in index:
    x,y,w,h = boxes_np[ind]
    bb_conf = int(confidences_np[ind]*100)
    classes_id = classes[ind]
    class_name = labels[classes_id]
    
    text = f"{class_name}: {bb_conf}%"
    cv2.rectangle(image, (x,y), (x+w, y+h), (255,0,0), 2)
    cv2.rectangle(image, (x,y-30), (x+int(0.35*w), y), (255,255,255), -1)
    
    cv2.putText(image, text, (x, y-10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0,0,0), 1)

In [12]:
# img = cv2.resize(img, (1200,1200))
# image = cv2.resize(image, (1200,1200))

# cv2.imshow('original', img)
# cv2.imshow('yolo_pred', image)

cv2.imwrite('save_yolo_pred_ap.jpg', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## LA

In [10]:
#load YAML
with open('LA_data.yaml', mode = 'r') as f:
    la_data_yaml = yaml.load(f, Loader = SafeLoader)
    
labels = la_data_yaml['names']
print(labels)

['APL1', 'APL2', 'APL3', 'APL4', 'APL5']


In [11]:
#load YOLO model
yolo = cv2.dnn.readNetFromONNX(r"E:\MAJOR_project_2\Dataset\BUU_LSPINE_400\BUU_LSPINE_400\LA_model\weights\best.onnx")
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [12]:
#load image
img = cv2.imread(r"E:\MAJOR_project_2\Dataset\BUU_LSPINE_400\BUU_LSPINE_400\LA\0001-F-037Y1.jpg")
image = img.copy()
row, col, d = image.shape

#get the yolo prediction from the image
#step-1: convert image into square image(array)
max_rc = max(row, col)
input_image = np.zeros((max_rc, max_rc, 3), dtype = np.uint8)
input_image[0:row, 0:col] = image

#step-2: get prediction from the square array
INPUT_WH_YOLO = 640
blob = cv2.dnn.blobFromImage(input_image, 1/255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB = True, crop = False)
yolo.setInput(blob)
preds = yolo.forward() # detection or prediction from yolo



# cv2.imshow('image', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [14]:
#non maximum supression
#step1: filter detection based on confidence(0.3) and probability score (0.2)
detections = preds[0]
boxes = []
confidences = []
classes = []

#width and height of image(input_image)
image_w, image_h = input_image.shape[0:2]
x_factor = image_w/INPUT_WH_YOLO
y_factor = image_h/INPUT_WH_YOLO

for i in range(len(detections)):
    row = detections[i]
    confidence = row[4]
    
    if confidence > 0.4:
        class_score = row[5:].max() #max probability from 5 objects
        class_id = row[5:].argmax()
        
        if class_score > 0.25:
            cx, cy, w, h = row[0:4]
            #construct bounding box from four values
            #left, top, width and height
            left = int((cx- 0.5*w)*x_factor)
            top = int((cy - 0.5*h)*y_factor)
            width = int(w*x_factor)
            height = int(h*y_factor)
            
            box = np.array([left, top, width, height])
            
            #append values into list
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)

#clean
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

#NMS
index = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.25, 0.4).flatten()


In [15]:
index

array([79,  9, 82, 21, 23])

In [16]:
#draw bounding box
for ind in index:
    x,y,w,h = boxes_np[ind]
    bb_conf = int(confidences_np[ind]*100)
    classes_id = classes[ind]
    class_name = labels[classes_id]
    
    text = f"{class_name}: {bb_conf}%"
    cv2.rectangle(image, (x,y), (x+w, y+h), (255,0,0), 2)
    cv2.rectangle(image, (x,y-30), (x+int(0.35*w), y), (255,255,255), -1)
    
    cv2.putText(image, text, (x, y-10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0,0,0), 1)

In [17]:
# img = cv2.resize(img, (1200,1200))
# image = cv2.resize(image, (1200,1200))

# cv2.imshow('original', img)
# cv2.imshow('yolo_pred', image)

cv2.imwrite('save_yolo_pred_la.jpg', image)
cv2.waitKey(0)
cv2.destroyAllWindows()